# Interactive Brain Visualization

In [1]:
print('hey')

hey


In [20]:
import neuropythy as ny
import ipyvolume as ipv
from matplotlib import cm
import numpy as np
import scipy.io
import os
import ipyvolume
import ipyvolume as ipv
#pip install --upgrade ipyvolume

In [21]:
local_path = '/Users/anna/Documents/Neurohackademy/test_subj'
if os.path.exists(local_path):
    sub = ny.freesurfer_subject(local_path)
    elecs = scipy.io.loadmat(os.path.join(local_path,'elecs/TDT_elecs_all.mat'))
else:
    print('file not found?')
    sub = ny.freesurfer_subject('test_subj')
    elecs=scipy.io.loadmat('/Applications/freesurfer_v5/subjects/test_subj/elecs/TDT_elecs_all.mat')


In [23]:
sub.hemis

lmap({'rhx': <lazy>, 'lhx': <lazy>, 'lh': <lazy>, 'rh': <lazy>})

In [30]:
ny.__version__


'0.9.8'

In [33]:
sub = ny.freesurfer_subject(local_path)
sub.rh.pial_surface

AttributeError: 'DataStruct' object has no attribute 'alt_label'

In [7]:
# printing the 3D brain
ipf = ny.cortex_plot(sub.rh.pial_surface, color=sub.rh.properties['BA44_weight'], cmap='Reds', alpha=0.7)
clr = np.ones((sub.rh.pial_surface.vertex_count,4))
clr[:,3] = 0.5
ipv.figure()
ipf = ny.cortex_plot(sub.rh.pial_surface, color=clr, alpha=0.5, underlay=None)

ipf.meshes[0].material.transparent = True
ipf.meshes[0].color = clr

elecs_scatter = ipv.scatter(elecs['elecmatrix'][:,0],elecs['elecmatrix'][:,1],elecs['elecmatrix'][:,2], marker='sphere', selected=[0,1,2,3,4])
elecs_scatter
ipv.selector_default() # for using selecting area, use control+L and select by mouse while holding on control

print(ipv.animation_control)


AttributeError: 'DataStruct' object has no attribute 'alt_label'

In [ ]:
from ipywidgets import FloatSlider, ColorPicker, VBox, jslink

size = FloatSlider(min=0, max=10, step=0.1)
size_selected = FloatSlider(min=0, max=10, step=0.1)
color = ColorPicker()
color_selected = ColorPicker()
jslink((elecs_scatter, 'size'), (size, 'value'))
jslink((elecs_scatter, 'size_selected'), (size_selected, 'value'))
jslink((elecs_scatter, 'color'), (color, 'value'))
jslink((elecs_scatter, 'color_selected'), (color_selected, 'value'))
VBox([ipv.gcc(), size, size_selected, color, color_selected])

ipv.show()


In [ ]:
elecs_scatter